In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
import pickle



In [64]:
#Load the trained model, scaler pickle, one hot encoding and label encoder

from tensorflow.keras.models import load_model

model = load_model('model.h5')

#loading the scaler and encoders now

with open('label_encoder_pickle.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

with open('ohe_pickle.pkl', 'rb') as file:
    ohe = pickle.load(file)

with open('Scaled_param.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [65]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [66]:
# One-hot encode the 'Geography' feature using the loaded OneHotEncoder
geo_encoded = ohe.transform([[input_data['Geography']]]).toarray()
geo_df = pd.DataFrame(geo_encoded, columns=ohe.get_feature_names_out(['Geography']))
geo_df

/Users/navdeepsingh/VS Code/env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [67]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [68]:
input_df['Gender']= label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [69]:
#concatenation

input_df = pd.concat([input_df.drop(['Geography'],axis=1),geo_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [70]:
#Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [73]:
#Prediction
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


array([[0.02410009]], dtype=float32)

In [78]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.02410009)

In [76]:
if(prediction_proba>0.5):
    print("The Customer will churn!")
else:
    print("The Customer will not churn!")

The Customer will not churn!
